In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))



True
NVIDIA GeForce GTX 1650


In [1]:
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare the dataset for training
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select([i for i in list(range(2000))])
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select([i for i in list(range(200))])

# Load the model and move it to the GPU
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased").to(device)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-distilbert")
tokenizer.save_pretrained("./fine-tuned-distilbert")

# Inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

# Test the model on a new text
text = "I feel the unusal suspects boring"
print(predict(text))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Viveka\AppData\Roaming\Python\Python39\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[codecarbon INFO @ 15:04:51] [setup] RAM Tracking...
[codecarbon INFO @ 15:04:51] [setup] GPU Tracking...
[codecarbon INFO @ 15:04:51] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:04:51] [setup] CPU Tracking...
[codecarbon WARNING @ 15:04:51] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:04:54] CPU Model on constant consumption mode: AMD Ryzen 5 460

Epoch,Training Loss,Validation Loss
1,No log,0.306705


[codecarbon INFO @ 15:05:09] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.764642238616943 W
[codecarbon INFO @ 15:05:09] Energy consumed for all GPUs : 0.000100 kWh. Total GPU Power : 24.057663900397593 W
[codecarbon INFO @ 15:05:09] Energy consumed for all CPUs : 0.000113 kWh. Total CPU Power : 27.0 W
[codecarbon INFO @ 15:05:09] 0.000237 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:05:24] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.764642238616943 W
[codecarbon INFO @ 15:05:24] Energy consumed for all GPUs : 0.000204 kWh. Total GPU Power : 24.86484759919545 W
[codecarbon INFO @ 15:05:24] Energy consumed for all CPUs : 0.000225 kWh. Total CPU Power : 27.0 W
[codecarbon INFO @ 15:05:24] 0.000477 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:05:39] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.764642238616943 W
[codecarbon INFO @ 15:05:39] Energy consumed for all GPUs : 0.000310 kWh. Total GPU Power : 25.5235148741585

Negative


In [7]:
# Inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

# Test the model on a new text
text = "Even though I like Schindler's List the holocaust scenes are depressing"
print(predict(text))

Negative
